# Some Unit tests for the HMM

In [1]:
import socket as socket
import os as os
import sys as sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM O2 Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/hapBLOCK/"  # The Path on Harvard Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd())

sys.path.append("./python3/") 
from main import HMM_Full
from plot.plot_posterior import plot_posterior, plot_posterior_panel

compute-a-16-151.o2.rc.hms.harvard.edu
HSM O2 Computational partition detected.
/n/groups/reich/hringbauer/git/hapBLOCK


# Test 1: HMM Likelihood on classic simulated example

In [8]:
def run_simulated_example():
    """The classic example"""
    h = HMM_Full(folder_in="./output/simulated/undermodel/sim2a/", l_model="simulated", t_model="standard", 
                     e_model="haploid_gl", h_model = "FiveStateFast", p_model="None",
                     output=True, load=True)
    h.t_obj.set_params(ibd_in = 0.0005, ibd_out = 0.001, ibd_jump = 0.05, max_gap=1.0)
    _, _, _, _, tot_ll = h.run_fwd_bwd()
    p=np.isclose(tot_ll, -26156.062, rtol=0, atol=0.01)
    return p

In [9]:
p = run_simulated_example()
assert(p)

Minimum Genetic Map: 0.0000 Morgan
Maximum Genetic Map: 9999.0000 Morgan
Gaps bigger than 0.1 cM: 9999
Maximum Gap: 100.0000 cM
Upper Gap Cutoff: 100.0000 cM
HMM State Number: 4
Memory Usage Full:
Memory Usage: 150.458368 mB
Total Log likelihood: -26156.062
